In [10]:
#HIDDEN
import pandas
import shap
import joblib
import ipywidgets as widgets
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches
from sklearn.base import TransformerMixin, BaseEstimator
from pandas import Categorical, get_dummies
import time
from ipywidgets import interact, interact_manual,interactive
from ipywidgets import Layout, Button, Box, VBox
from ipywidgets import Button, HBox, VBox
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider
import warnings
warnings.filterwarnings("ignore")
#HIDDEN
class CategoricalPreprocessing(BaseEstimator, TransformerMixin):
    def __get_categorical_variables(self,data_frame,threshold=0.70, top_n_values=10):
        likely_categorical = []
        for column in data_frame.columns:
            if 1. * data_frame[column].value_counts(normalize=True).head(top_n_values).sum() > threshold:
                likely_categorical.append(column)
        try:
            likely_categorical.remove('st_depression')
        except:
            pass 
        return likely_categorical
    
    def fit(self, X, y=None):
        self.attribute_names = self.__get_categorical_variables(X)
        cats = {}
        for column in self.attribute_names:
            cats[column] = X[column].unique().tolist()
        self.categoricals = cats
        return self

    def transform(self, X, y=None):
        df = X.copy()
        for column in self.attribute_names:
            df[column] = Categorical(df[column], categories=self.categoricals[column])
        new_df = get_dummies(df, drop_first=False)
        # in case we need them later
        return new_df
#HIDDEN
feature_model=joblib.load('random_forest_heart_model_v2')
categorical_transform=joblib.load('categorical_transform_v2')
explainer_random_forest=joblib.load('shap_random_forest_explainer_v2')
numerical_options=joblib.load('numerical_options_dictionary_v2')
categorical_options=joblib.load('categorical_options_dictionary_v2')


ui_elements=[]
style = {'description_width': 'initial'}
for i in numerical_options.keys():
    minimum=numerical_options[i]['minimum']
    maximum=numerical_options[i]['maximum']
    default=numerical_options[i]['default']
    if i!='st_depression':
        ui_elements.append(widgets.IntSlider(
        value=default,
        min=minimum,
        max=maximum,
        step=1,
        description=i,style=style)
                      )
    else:
        ui_elements.append(widgets.FloatSlider(
        value=default,
        min=minimum,
        max=maximum,
        step=.5,
        description=i,style=style)
                      )
for i in categorical_options.keys():
    ui_elements.append(widgets.Dropdown(
    options=categorical_options[i]['options'],
    value=categorical_options[i]['default'],
    description=i,style=style
    ))
interact_calc=interact.options(manual=True, manual_name="Calculate Risk")

#HIDDEN
def get_risk_string(prediction_probability):
    y_val = prediction_probability* 100
    text_val = str(np.round(y_val, 1)) + "% | "

    # assign a risk group
    if y_val / 100 <= 0.275685:
        risk_grp = ' low risk '
    elif y_val / 100 <= 0.795583:
        risk_grp = ' medium risk '
    else:
        risk_grp = ' high risk '
    
    return text_val+ risk_grp
def get_current_prediction():
    current_values=dict()
    for element in ui_elements:
        current_values[element.description]=element.value
    feature_row=categorical_transform.transform(pandas.DataFrame.from_dict([current_values]))
    feature_row=feature_row[['age', 'resting_blood_pressure', 'cholesterol',
       'max_heart_rate_achieved', 'st_depression', 'sex_female', 'sex_male',
       'chest_pain_type_non-anginal pain', 'chest_pain_type_asymptomatic',
       'chest_pain_type_atypical angina', 'chest_pain_type_typical angina',
       'fasting_blood_sugar_0', 'fasting_blood_sugar_1', 'rest_ecg_normal',
       'rest_ecg_ST-T wave abnormality',
       'rest_ecg_left ventricular hypertrophy', 'exercise_induced_angina_0',
       'exercise_induced_angina_1', 'st_slope_flat', 'st_slope_upsloping',
       'st_slope_downsloping']].copy()
    
   
    
    shap_values = explainer_random_forest.shap_values(feature_row)
    
   
    
    
    updated_fnames = feature_row.T.reset_index()
    updated_fnames.columns = ['feature', 'value']
    
    
    risk_prefix='<h2> Risk Level :'
    risk_suffix='</h2>'
    risk_probability=feature_model.predict_proba(feature_row)[0][1]
    risk_string=get_risk_string(risk_probability)
    risk_widget.value=risk_prefix+risk_string+risk_suffix
    updated_fnames['shap_original'] = pandas.Series(shap_values[1][0])
    updated_fnames['shap_abs'] = updated_fnames['shap_original'].abs()

    updated_fnames=updated_fnames[updated_fnames['value']!=0]
    
    
    
    plt.rcParams.update({'font.size': 30})
    df1=pandas.DataFrame(updated_fnames["shap_original"])
    df1.index=updated_fnames.feature
    df1.sort_values(by='shap_original',inplace=True,ascending=True)
    df1['positive'] = df1['shap_original'] > 0
    df1.plot(kind='barh',figsize=(15,7,),legend=False)
#HIDDEN
form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

form_items = ui_elements

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
   
    align_items='stretch'
   
))


box_layout = Layout(display='flex',
                    flex_flow='column'
                    )

left_box = VBox(ui_elements[0:5])
right_box = VBox(ui_elements[5:])
control_layout=VBox([left_box,right_box],layout=box_layout)

risk_string="<h2>Risk Level</h2>"

risk_widget=widgets.HTML(value=risk_string)



In [13]:
#HIDDEN
display(control_layout)
display(Box(children=[risk_widget]))
risk_plot=interact_calc(get_current_prediction)
risk_plot.widget.layout=Layout(width='60%'
                    )

Box(children=(HTML(value='<h2> Risk Level :91.4% |  high risk </h2>'),))

interactive(children=(Button(description='Calculate Risk', style=ButtonStyle()), Output()), _dom_classes=('wid…